## 爬取掌阅书籍的特价信息
2018-08-08

In [ ]:
import requests
import pandas as pd
import time
from lxml import etree

In [ ]:
# cid type
# 93  小说
# 113 文学
# 142 历史
# 179 社科
# 168 心理

In [ ]:
cids = [93,113,142,179,168]
urls = []
for cid in cids:
    # 出版pid=92，特价status=2
    url = 'http://www.ireader.com/index.php?ca=booksort.index&pca=channel.index&pid=92&cid='+str(cid)+'&order=download&status=2&page='
    print('cid:'+str(cid),end=' ')
    # 获取每一类别的前5页的书籍都URL
    for i in range(5):
        result = requests.get(url+str(i))
        selector = etree.HTML(result.content)
        hrefs = selector.xpath('//div[@class="show"]/ul/li/a/@href')
        urls.extend(hrefs)
        print(i,end=' ')
    print()
    # 去重
    book_urls = list(set(urls))
    book_urls.sort(key = urls.index)

names = []
scores = []
prices = []
origs = []

i = 0
print(len(book_urls))
for book_url in book_urls:
    # 获取书籍页信息
    book = requests.get(book_url)
    book_selector = etree.HTML(book.text)

    name = book_selector.xpath('//div[@class="bookname"]/h2/a/text()')[0]
    # 部分书籍没有评分
    try:
        score = book_selector.xpath('//div[@class="bookname"]/span/text()')[0]
    except:
        score = 0
    # 由于我选择的是特价页面，价格和原价都不为0
    price = book_selector.xpath('//div[@class="left"]/p/i[@class="price"]/text()')[0]
    orig = book_selector.xpath('//div[@class="left"]/p/i[@class="orig"]/text()')[0]

    names.append(name)
    scores.append(score)
    prices.append(price)
    origs.append(orig)
    i += 1
    if (i%10==0):
        print(i,end=' ')
    
# 生成DataFrame
book_dic = {'name':names, 'score':scores, 'price':prices, 'orig':origs}
book_df = pd.DataFrame(book_dic)

# 提取出价格和原始价格，并计算折扣
book_df['price_num'] = book_df['price'].str.extract(r'(\d+)', expand=False).astype(int)
book_df['orig_num'] = book_df['orig'].str.extract(r'(\d+)', expand=False).astype(int)
book_df['discount'] = book_df['price_num']/book_df['orig_num']

t = book_df.sort_values(['discount'])

# 以当天的日期作为文件名，保存DataFrame
time_now = time.strftime("%Y-%m-%d", time.localtime())
t.to_csv('./'+time_now+'.csv')
         
# 输出价格小于200月饼的前100本书
pd.set_option('display.max_rows',100)
t[t['price_num']<=200].head(100)

In [ ]:
t1 = t.sort_values(['score','discount'],ascending=[False,True])
t1[t1['price_num']<=100].head(100)